In [52]:
from flask import Flask, request, jsonify
import pandas as pd
from prophet import Prophet
import logging
from google.colab import output
import threading

app = Flask(__name__)

# Enable logging
logging.basicConfig(level=logging.INFO)

# Load dataset
data = pd.read_csv('synthetic_data.csv')
data['Date'] = pd.to_datetime(data['Date'], format='mixed', errors='coerce')

# Create a dictionary for installed capacity lookup
installed_capacity = data[['District', 'Installed Capacity (MW)']].drop_duplicates().set_index('District').to_dict()['Installed Capacity (MW)']

# Train Prophet models for each district
models = {}

for district in data['District'].unique():
    district_data = data[data['District'] == district]

    # Load Demand Model
    load_model = Prophet()
    load_model.add_regressor('Price (₹/unit)')
    load_model.add_regressor('Installed Capacity (MW)')
    load_model.fit(district_data.rename(columns={'Date': 'ds', 'Load Demand (MW)': 'y'}))

    # Price Model
    price_model = Prophet()
    price_model.add_regressor('Load Demand (MW)')
    price_model.fit(district_data.rename(columns={'Date': 'ds', 'Price (₹/unit)': 'y'}))

    # Blackout Risk Model
    blackout_model = Prophet()
    blackout_model.add_regressor('Load Demand (MW)')
    blackout_model.add_regressor('Installed Capacity (MW)')
    blackout_model.fit(district_data.rename(columns={'Date': 'ds', 'Blackout Risk (%)': 'y'}))

    models[district] = {
        'load': load_model,
        'price': price_model,
        'blackout': blackout_model
    }

@app.route('/predict', methods=['GET'])
def predict():
    district = request.args.get('district')
    future_date = request.args.get('date')

    if district not in models:
        return jsonify({'error': 'Invalid district'}), 400

    if not future_date:
        return jsonify({'error': 'Missing future date'}), 400

    try:
        future_date = pd.to_datetime(future_date)

        # Prepare future dataframe
        future = pd.DataFrame({'ds': [future_date]})

        # Installed capacity for the district
        installed_capacity_value = installed_capacity.get(district, None)

        # Predict Load Demand
        future['Price (₹/unit)'] = data.loc[data['District'] == district, 'Price (₹/unit)'].mean()
        future['Installed Capacity (MW)'] = installed_capacity_value
        load_pred = models[district]['load'].predict(future)['yhat'].iloc[0]

        # Predict Price
        future['Load Demand (MW)'] = load_pred
        price_pred = models[district]['price'].predict(future)['yhat'].iloc[0]

        # Predict Blackout Risk
        future['Load Demand (MW)'] = load_pred
        future['Installed Capacity (MW)'] = installed_capacity_value
        blackout_pred = models[district]['blackout'].predict(future)['yhat'].iloc[0]

        response = {
            'District': district,
            'Date': future_date.strftime('%Y-%m-%d'),
            'Load Demand (MW)': round(load_pred, 2),
            'Price (₹/unit)': round(price_pred, 2),
            'Blackout Risk (%)': round(blackout_pred, 2),
            'Installed Capacity (MW)': installed_capacity_value
        }

        return jsonify(response)

    except Exception as e:
        logging.error(f"Prediction error: {e}")
        return jsonify({'error': 'Prediction failed', 'message': str(e)}), 500


if __name__ == '__main__':
    threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000)).start()

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpi9kua17m/dh8n9_em.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpi9kua17m/a1hfwxcx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=23385', 'data', 'file=/tmp/tmpi9kua17m/dh8n9_em.json', 'init=/tmp/tmpi9kua17m/a1hfwxcx.json', 'output', 'file=/tmp/tmpi9kua17m/prophet_model9bphsn6i/prophet_model-20250318220930.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
22:09:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:09:30 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonali

 * Serving Flask app '__main__'
 * Debug mode: off


In [19]:
!pip install flask_ngrok


In [42]:
!pip install flask_cors


In [11]:
!ngrok authtoken 2tzIAXowZtLVkWoorlhV8L8L7ul_h2xcxt2mZ1HG2rJ39d61

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
!pip install pyngrok

In [3]:
import requests

def get_prediction(district, date):
    url = f"http://127.0.0.1:5000/predict?district={district}&date={date}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        print("\n✅ Prediction Result:")
        print(result)
    else:
        print(f"\n❌ Error: {response.json().get('error', 'Unknown error')}")

In [57]:
get_prediction('Kolkata', '2025.03.20')


INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 22:30:15] "GET /predict?district=Kolkata&date=2025.03.20 HTTP/1.1" 200 -



✅ Prediction Result:
{'Blackout Risk (%)': 192.81, 'Date': '2025-03-20', 'District': 'Kolkata', 'Installed Capacity (MW)': 175.0, 'Load Demand (MW)': 639.37, 'Price (₹/unit)': 1.42}


In [1]:
!pip install flask pandas numpy prophet
